# Defintions

In [1]:
import numpy as np
import pandas as pd
from importlib import reload
from scipy.optimize import least_squares, minimize
import sys

In [2]:
sys.path.append('/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial/Monogem_on_off_bkg/Hess_1/Detection/')
import runmatching_utilities as util

In [3]:
sys.path.insert(1, '/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial')
import my_utility_functions as muf

In [4]:
import sys
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from pathlib import Path
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, RectangleSkyRegion
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.data import DataStore
from gammapy.datasets import Datasets, FluxPointsDataset, MapDataset
from gammapy.modeling import Fit
from gammapy.modeling.models import SkyModel, LogParabolaSpectralModel, PointSpatialModel, PowerLawSpectralModel, FoVBackgroundModel, Models
from gammapy.estimators import FluxPoints,FluxPointsEstimator, ExcessMapEstimator
import sys, os
import numpy as np
import itertools
import re
import datetime
from datetime import datetime
import gammapy
import time
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import json
from pathlib import Path
from astropy import units as u
from astropy.coordinates import SkyCoord
from gammapy.stats import CashCountsStatistic, cash, cash_sum_cython
from gammapy.maps import WcsGeom, MapAxis, Map
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.data import DataStore, Observations
from gammapy.datasets import Datasets, FluxPointsDataset, MapDatasetOnOff
from gammapy.modeling import Fit
from gammapy.modeling.models import SkyModel, LogParabolaSpectralModel, PointSpatialModel, PowerLawSpectralModel, DiskSpatialModel, TemplateSpatialModel, PowerLawNormSpectralModel
from gammapy.datasets import Datasets, MapDataset
from regions import CircleSkyRegion, RectangleSkyRegion, EllipseSkyRegion
from gammapy.estimators import (
    FluxPoints,
    FluxPointsEstimator,
    EnergyDependentMorphologyEstimator)
from gammapy.modeling.models import FoVBackgroundModel, GaussianSpatialModel, ShellSpatialModel, DiskSpatialModel, PiecewiseNormSpectralModel
from gammapy.makers import FoVBackgroundMaker
from gammapy.estimators import ExcessMapEstimator, TSMapEstimator, FluxMaps
from gammapy.maps import WcsNDMap
from gammapy.visualization import plot_npred_signal, plot_distribution
import ipywidgets
from gammapy.modeling.models import Models
from astropy.coordinates import Angle
from collections import OrderedDict
import glob
import matplotlib.pyplot as plt
from scipy.stats import chi2, norm
from astropy.table import Table
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
sys.path.insert(1, '/home/wecapstor1/caph/caph101h/projects/test_projects/HAWC_J0543+233/python_scripts/')
DB_general = pd.read_csv('/home/wecapstor1/caph/caph101h/projects/test_projects/Run_matching_tutorial/db_data.csv', header=0)
from scipy.optimize import curve_fit
from matplotlib.lines import Line2D
from PIL import Image
import run_matching_algorithm as rma
import runmatching_utilities as util

# Matching

In [5]:
# Getting the Off run database
off_runs = util.query_off_runs(FitsEra='HESS1')
quality_selection = util.quality_selection(runlist=off_runs, HessEra='HESS1', Config='stereo', FitsEra='HESS1', AnalysisType='spectral')
off_runs_filtered = quality_selection.quality_cuts(requireCT5=False, logBadRuns=False)
print(len(off_runs_filtered))

5568


In [15]:
# Loading your On run list
outputBase = '/home/vault/caph/caph101h/datasets/2FGL_J1044_point_5_5737/'
all_on_runs = np.loadtxt('{}/on_run_list.txt'.format(outputBase), usecols=(0,) , dtype=float)
all_on_runs_list = all_on_runs.tolist()
print(len(all_on_runs_list))

73


In [12]:
# Add here a save path where you want your runlist saved
save_path_on_off = '/home/vault/caph/caph101h/datasets/2FGL_J1044_point_5_5737/individual_matching/datasets/'

for runs in all_on_runs_list:
    print(f'Time: {datetime.now().time()}, Begin match process for Obs-Id: {int(runs)}')
    # Initialize the list to store all matched Off runs for this On run
    matched_off_run_list = []
    found_new_matches = True

    # Parameters for the run matching, do not go higher than that!
    Fits_Era = 'HESS1'
    duration = 0.25 # 0.25
    nsb = 0.9 # 0.9
    muon_eff = 0.19 # 0.19 max
    transparency = 0.19 # 0.19 max
    trig_rate = 0.45 # 0.45
    radio = 0.55 # 0.55
    
    while found_new_matches:
        # Get the Off runs from the quality cuts applied above
        off_runs_filtered_1 = off_runs_filtered.copy()
        off_runs_filtered_1 = [item[0] for item in off_runs_filtered_1]

        off_runs_filtered_1.remove(21211)
        off_runs_filtered_1.remove(21212)
        off_runs_filtered_1.remove(21213)
        off_runs_filtered_1.remove(21214)
        off_runs_filtered_1.remove(23702)
        off_runs_filtered_1.remove(24472)
        off_runs_filtered_1.remove(31528)

        # print(f'{len(off_runs_filtered_1)} Off runs available')
     
        # Remove already matched Off runs to avoid duplications
        for matched in matched_off_run_list:
            off_runs_filtered_1.remove(matched[0][0]) 
        
        run_name = 'Obs_id_{}'.format(int(runs))
        
        # OnRuns must be a list so this is not nice, but it works
        on_runs = [runs]
        
        # Perform the run matching
        RM = util.run_matching(OnRuns=on_runs, OffRuns=off_runs_filtered_1, FitsEra=Fits_Era,
                               duration=duration,
                               nsb=nsb,
                               muon_eff=muon_eff,
                               transparency=transparency,
                               trig_rate=trig_rate,
                               radio=radio)
        
        matches = RM.matching_operation()

        matched_on_run = [item[0] for item in matches]
        matched_off_run = [item[1] for item in matches]
        deviation = [item[2] for item in matches]

        # Add the matched Off runs until there are none
        if len(matched_off_run) != 0:
            matched_off_run_list.append((matched_off_run, deviation))
        else:
            found_new_matches = False
            print(f'No more matches found for Obs-Id: {int(runs)}')

    # Save the final list of matched Off runs after all possible matches are found   
    converted_array = np.array([[x[0][0], x[1][0]] for x in matched_off_run_list])
    np.savetxt('{}/Off_run_list_for_on_run_Id_{}.txt'.format(save_path_on_off, run_name), converted_array) 
    
    print(f'Time: {datetime.now().time()}, Finished match process for Obs-Id: {int(runs)}, {len(converted_array)} Off runs found')
    print('\n')

    import gc
    gc.collect()

Time: 09:56:11.992819, Begin match process for Obs-Id: 61705
No more matches found for Obs-Id: 61705
Time: 09:56:26.144491, Finished match process for Obs-Id: 61705, 5 Off runs found


Time: 09:56:26.232514, Begin match process for Obs-Id: 61706
No more matches found for Obs-Id: 61706
Time: 09:56:29.011039, Finished match process for Obs-Id: 61706, 0 Off runs found


Time: 09:56:29.091017, Begin match process for Obs-Id: 61751
No more matches found for Obs-Id: 61751
Time: 10:01:19.084106, Finished match process for Obs-Id: 61751, 46 Off runs found


Time: 10:01:19.169680, Begin match process for Obs-Id: 61752
No more matches found for Obs-Id: 61752
Time: 10:01:33.118361, Finished match process for Obs-Id: 61752, 5 Off runs found


Time: 10:01:33.192810, Begin match process for Obs-Id: 61753
No more matches found for Obs-Id: 61753
Time: 10:01:47.150050, Finished match process for Obs-Id: 61753, 5 Off runs found


Time: 10:01:47.219028, Begin match process for Obs-Id: 61763
No more match

In [13]:
print('\n')
print('stop')
print('\n')



stop


